## 기업 코드 적재

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import create_engine, text
import sys
sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv()

DB_USER = os.getenv("POSTGRE_USER")
DB_PASSWORD = os.getenv("POSTGRE_PASSWORD")
DB_HOST = os.getenv("POSTGRE_HOST", "localhost")
DB_PORT = os.getenv("POSTGRE_PORT", "5432")
DB_NAME = os.getenv("POSTGRE_DB")

db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

# 기업 코드 수집 모듈
from app.crawler.dart_loader import get_corp_code_mapping

create_table_sql = """
CREATE TABLE IF NOT EXISTS dart_corp_map (
    corp_code TEXT PRIMARY KEY,
    corp_name TEXT,
    stock_name TEXT
);
"""

with engine.begin() as conn:
    conn.execute(text(create_table_sql))

### 1. 기업 추가

In [8]:
corp_names = ['NAVER', 
              '035720', # 카카오
              '당근마켓', 
              '당근페이', 
              '삼성에스디에스', 
              'LG씨엔에스'
             ]

,corp_code,corp_name,stock_name
0,00266961,네이버(주),NAVER
1,00258801,(주)카카오,카카오
2,01547845,주식회사 당근마켓,당근마켓
3,01717824,주식회사 당근페이,당근페이
4,00126186,삼성에스디에스(주),삼성에스디에스
5,00139834,(주)엘지씨엔에스,LG씨엔에스


### 2. 코드 매핑

In [ ]:
df = get_corp_code_mapping(corp_names)
df

### 3. 적재

In [9]:
df.to_sql(
    'dart_corp_map',
    con=engine,
    if_exists='replace',  # append: 데이터 추가, replace: 전체 덮어쓰기
    index=False
)

6

### 4. 적재 확인

In [10]:
query = "SELECT * FROM dart_corp_map;"
check = pd.read_sql(query, engine)
check

,corp_code,corp_name,stock_name
0,00266961,네이버(주),NAVER
1,00258801,(주)카카오,카카오
2,01547845,주식회사 당근마켓,당근마켓
3,01717824,주식회사 당근페이,당근페이
4,00126186,삼성에스디에스(주),삼성에스디에스
5,00139834,(주)엘지씨엔에스,LG씨엔에스
